# MIGHT example on Wise-1 cancer data

## Import libraries and local functions

In [1]:
# dependency libararies
import warnings
import pickle
import time
import numpy as np
import pandas as pd

# treeple functions
from treeple.ensemble import HonestForestClassifier
from treeple.tree import ObliqueDecisionTreeClassifier
from treeple.stats import build_oob_forest

warnings.filterwarnings("ignore")

In [2]:
# local function for processing cancer data files
def get_X_y(f, root="./data/", cohort=[], verbose=False):
    df = pd.read_csv(root + f)
    non_features = [
        "Run",
        "Sample",
        "Library",
        "Cancer Status",
        "Tumor type",
        "Stage",
        "Library volume (uL)",
        "Library Volume",
        "UIDs Used",
        "Experiment",
        "P7",
        "P7 Primer",
        "MAF",
    ]
    sample_ids = df["Sample"]
    # if sample is contains "Run" column, remove it
    for i, sample_id in enumerate(sample_ids):
        if "." in sample_id:
            sample_ids[i] = sample_id.split(".")[1]
    target = "Cancer Status"
    y = df[target]
    # convert the labels to 0 and 1
    y = y.replace("Healthy", 0)
    y = y.replace("Cancer", 1)
    # remove the non-feature columns if they exist
    for col in non_features:
        if col in df.columns:
            df = df.drop(col, axis=1)
    nan_cols = df.isnull().all(axis=0).to_numpy()
    # drop the columns with all nan values
    df = df.loc[:, ~nan_cols]
    # if cohort is not None, filter the samples
    if cohort is not None:
        # filter the rows with cohort1 samples
        X = df[sample_ids.isin(cohort)]
        y = y[sample_ids.isin(cohort)]
    else:
        X = df
    if "Wise" in f:
        # replace nans with zero
        X = X.fillna(0)
    # impute the nan values with the mean of the column
    X = X.fillna(X.mean(axis=0))
    # check if there are nan values
    # nan_rows = X.isnull().any(axis=1)
    nan_cols = X.isnull().all(axis=0)
    # remove the columns with all nan values
    X = X.loc[:, ~nan_cols]
    if verbose:
        if nan_cols.sum() > 0:
            print(f)
            print(f"nan_cols: {nan_cols.sum()}")
            print(f"X shape: {X.shape}, y shape: {y.shape}")
        else:
            print(f)
            print(f"X shape: {X.shape}, y shape: {y.shape}")
    # X = X.dropna()
    # y = y.drop(nan_rows.index)

    return X, y

## Process the Wise-1 cancer data

In [3]:
DIRECTORY = "./"
N_JOBS = 4
N_EST = 100 # 100_000

# collect sample data
sample_list_file = DIRECTORY + "AllSamples.MIGHT.Passed.samples.txt"
sample_list = pd.read_csv(sample_list_file, sep=" ", header=None)
sample_list.columns = ["library", "sample_id", "cohort"]

# get the sample ids for specific cohorts
cohort1 = sample_list[sample_list["cohort"] == "Cohort1"]["sample_id"]
cohort2 = sample_list[sample_list["cohort"] == "Cohort2"]["sample_id"]
PON = sample_list[sample_list["cohort"] == "PanelOfNormals"]["sample_id"]

In [4]:
# obtain the samples for cohort 1
X, y = get_X_y("WiseCondorX.Wise-1.csv.gz", root=DIRECTORY, cohort=cohort1)

# Save the processed data to a CSV file for use in the Yggdrasil notebook
processed_data = X.copy()
processed_data["Cancer Status"] = y
processed_data.to_csv("processed_wise1_data.csv", index=False)

In [5]:
X.shape

(352, 2523)

## Run axis-aligned MIGHT

In [41]:
est = HonestForestClassifier(
    n_estimators=N_EST,
    max_samples=1.6,
    max_features=0.3,
    bootstrap=True,
    stratify=True,
    n_jobs=N_JOBS,
    random_state=23,
    honest_prior="ignore",
    honest_method='prune',
)

# record start time
start_time = time.perf_counter()

# obtain tree level posteriors
fitted_est, tree_proba = build_oob_forest(est, X, y)

# calculate fitting time
end_time = time.perf_counter()
time_dif = end_time-start_time

print(time_dif)

1.2514145409986668


In [42]:
# obtain forest level posteriors
forest_proba = np.nanmean(tree_proba, axis=0)

## Or run oblique MIGHT

In [1]:
# dependency libararies
import warnings
import pickle
import time
import numpy as np
import pandas as pd

# treeple functions
from treeple.ensemble import HonestForestClassifier
from treeple.tree import ObliqueDecisionTreeClassifier
from treeple.stats import build_oob_forest

warnings.filterwarnings("ignore")

In [2]:
est = HonestForestClassifier(
    n_estimators=1000,
    max_samples=1.0, # Use all samples for bootstrapping. Use 1.0 instead of 1. 1 means literally 1 sample
    max_features=128, # Keep this static
    bootstrap=True,
    stratify=True,
    n_jobs=-1, # Use all resources
    random_state=42,
    honest_prior="ignore",
    honest_method='prune',
    tree_estimator=ObliqueDecisionTreeClassifier(),
    max_depth = None, # Fully grow trees
    n_threads = 96
)

# # record start time
# start_time = time.perf_counter()

# # obtain tree level posteriors
# fitted_est, tree_proba = build_oob_forest(est, X, y)

# # calculate fitting time
# end_time = time.perf_counter()
# time_dif = end_time-start_time

# print(time_dif)

## Benchmarks

In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# MIGHT / treeple
from treeple.ensemble import HonestForestClassifier
from treeple.tree import ObliqueDecisionTreeClassifier
from treeple.stats import build_oob_forest


def benchmark_might(n_vals, d_vals, repeats=7, random_state=42, log_file_path=None):  # NEW: pass path
    """
    Benchmarks MIGHT's training & inference time across multiple (n, d) data sizes,
    repeating each (n,d) `repeats` times to compute mean & std dev.

    Returns a DataFrame with columns:
      ["n", "d",
       "train_time_mean", "train_time_std",
       "inference_time_mean", "inference_time_std",
       "accuracy_mean", "accuracy_std"]
    """
    np.random.seed(random_state)
    results = []

    for n in n_vals:
        for d in d_vals:
            train_times = []
            inference_times = []
            accuracies = []

            for _ in range(repeats):
                # 1) Generate random data
                X = np.random.randn(n, d)
                y = np.random.randint(2, size=n)

                # 2) Split
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y, test_size=0.3, random_state=random_state, stratify=y
                )

                # 3) Convert to DataFrame
                df_train = pd.DataFrame(X_train)
                df_test  = pd.DataFrame(X_test)

                # 4) Build classifier
                # You could define this earlier, or globally, etc.
                global est

                # 5) Training time
                t0 = time.perf_counter()
                fitted_est, tree_proba = build_oob_forest(est, df_train, y_train)
                t1 = time.perf_counter()
                train_times.append(t1 - t0)

                # 6) Inference time
                t2 = time.perf_counter()
                test_tree_proba = []
                for tree in fitted_est.estimators_:
                    p = tree.predict_proba(df_test)  # shape (len(X_test), 2)
                    test_tree_proba.append(p)
                test_tree_proba = np.stack(test_tree_proba, axis=0)
                avg_proba = test_tree_proba.mean(axis=0)
                y_proba_test = avg_proba[:, 1]
                y_pred_test = (y_proba_test >= 0.5).astype(int)
                t3 = time.perf_counter()
                inference_times.append(t3 - t2)

                # 7) Accuracy
                acc = accuracy_score(y_test, y_pred_test)
                accuracies.append(acc)

            # Mean/std
            train_time_mean = np.mean(train_times)
            train_time_std  = np.std(train_times)
            inf_time_mean   = np.mean(inference_times)
            inf_time_std    = np.std(inference_times)
            acc_mean        = np.mean(accuracies)
            acc_std         = np.std(accuracies)

            msg = (
                f"\n(n={n}, d={d}):\n"
                f"  train_time mean={train_time_mean:.4f}s, std={train_time_std:.4f}\n"
                f"  inference_time mean={inf_time_mean:.4f}s, std={inf_time_std:.4f}\n"
                f"  accuracy mean={acc_mean:.3f}, std={acc_std:.3f}"
            )

            # Print to screen
            print(msg)
            
            # Write to file right away, no buffering
            if log_file_path:  # NEW
                with open(log_file_path, "a") as f:  # open in append mode
                    f.write(msg + "\n")
                    f.flush()  # force flush, ensures content is written

            results.append({
                "n": n,
                "d": d,
                "train_time_mean": train_time_mean,
                "train_time_std": train_time_std,
                "inference_time_mean": inf_time_mean,
                "inference_time_std": inf_time_std,
                "accuracy_mean": acc_mean,
                "accuracy_std": acc_std,
            })

    return pd.DataFrame(results)


if __name__ == "__main__":

    n_values = [256,512,1024,2048,4096,8192, 16384, 32768, 65536, 131072]
    d_values = [128,256,512,1024,2048,4096,8192, 16384, 32768, 65536, 131072]

    # If you want a header first, you can do that outside:
    with open("might_benchmark_output.txt", "w") as f:
        f.write("=== MIGHT Benchmark Output ===\n")
        f.flush()

    df_bench = benchmark_might(n_values, d_values, repeats=3,
                               random_state=42, log_file_path="might_benchmark_output.txt")

    # Final summary
    summary = "\n=== MIGHT Benchmark Results (Averaged Over Repeats) ===\n"
    print(summary)
    with open("might_benchmark_output.txt", "a") as f:
        f.write(summary + "\n")
        f.flush()

    print(df_bench)
    with open("might_benchmark_output.txt", "a") as f:
        f.write(df_bench.to_string(index=False) + "\n")
        f.flush()


(n=256, d=128):
  train_time mean=5.7586s, std=0.0387
  inference_time mean=1.0177s, std=0.0020
  accuracy mean=0.515, std=0.058

(n=256, d=256):
  train_time mean=5.8208s, std=0.0345
  inference_time mean=1.4511s, std=0.0024
  accuracy mean=0.541, std=0.012

(n=256, d=512):
  train_time mean=5.9991s, std=0.0835
  inference_time mean=2.3131s, std=0.0065
  accuracy mean=0.519, std=0.000

(n=256, d=1024):
  train_time mean=6.0633s, std=0.0238
  inference_time mean=3.9988s, std=0.0046
  accuracy mean=0.537, std=0.016
